# Production Planning with Two-Stage Stochastic Programming

## Problem Statement

This code solves a production planning problem under demand uncertainty using a two-stage stochastic programming approach. In this problem:

1. A manufacturer must decide how much of a product to produce **before** knowing the actual demand (first-stage decision).
2. After the demand is revealed, the manufacturer must decide how much to sell, how much to store, and how much shortage to incur (second-stage decisions).
3. The goal is to minimize the total expected cost across all possible demand scenarios.

## Required Components

### Libraries
- **NumPy**: For random number generation and array operations
- **Pyomo**: For mathematical optimization modeling
- **GLPK**: The solver used to solve the linear programming model (must be installed separately)

### Problem Parameters
1. **Production Cost**: $10 per unit
2. **Storage Cost**: $2 per unit
3. **Shortage Cost**: $20 per unit (penalty for not meeting demand)
4. **Selling Price**: $15 per unit
5. **Base Demand**: 100 units
6. **Demand Uncertainty**: Normally distributed with mean 0 and standard deviation 20

## Code Walkthrough

### Random Scenario Generation
```python
# Generate random demand scenarios
base_demand = 100
demand_scenarios = base_demand + np.random.normal(0, 20, num_scenarios)
scenario_probs = np.ones(num_scenarios) / num_scenarios  # Equal probability
```

This code generates `num_scenarios` different potential demand values. Each scenario represents a possible future demand realization. The demands follow a normal distribution with mean 100 and standard deviation 20. Each scenario is assigned equal probability.

### Model Structure

#### Decision Variables
```python
# First stage decision variable: how much to produce
model.produce = pyo.Var(domain=pyo.NonNegativeReals)

# Second stage variables - for each scenario
model.scenarios = pyo.RangeSet(0, num_scenarios-1)
model.sell = pyo.Var(model.scenarios, domain=pyo.NonNegativeReals)
model.store = pyo.Var(model.scenarios, domain=pyo.NonNegativeReals)
model.shortage = pyo.Var(model.scenarios, domain=pyo.NonNegativeReals)
```

- **First Stage**: `model.produce` (before knowing demand)
- **Second Stage** (for each scenario):
  - `model.sell[s]`: Amount to sell in scenario s
  - `model.store[s]`: Amount to store in scenario s
  - `model.shortage[s]`: Amount of unmet demand in scenario s

#### Objective Function
```python
# First stage objective: minimize production cost
model.first_stage_cost = pyo.Expression(expr=production_cost * model.produce)

# Second stage objective: expected profit/cost across all scenarios
def second_stage_rule(model, s):
    return (scenario_probs[s] * 
            (selling_price * model.sell[s] - 
             storage_cost * model.store[s] - 
             shortage_cost * model.shortage[s]))

model.second_stage_obj = pyo.Expression(model.scenarios, rule=second_stage_rule)

# Total objective: first stage cost + expected second stage cost
model.obj = pyo.Objective(
    expr=model.first_stage_cost - sum(model.second_stage_obj[s] for s in model.scenarios),
    sense=pyo.minimize
)
```

The objective combines:
1. **First-stage cost**: Production cost of units made
2. **Expected second-stage cost**: Weighted average of revenue minus costs across all scenarios
   - Revenue from selling units
   - Minus storage costs for unsold units
   - Minus shortage penalties for unmet demand

#### Constraints
```python
# For each scenario, amount sold + stored equals amount produced
def balance_rule(model, s):
    return model.sell[s] + model.store[s] == model.produce

model.balance = pyo.Constraint(model.scenarios, rule=balance_rule)

# For each scenario, amount sold + shortage equals demand
def demand_rule(model, s):
    return model.sell[s] + model.shortage[s] == demand_scenarios[s]

model.demand = pyo.Constraint(model.scenarios, rule=demand_rule)
```

Two key constraints:
1. **Balance constraint**: The sum of units sold and stored must equal the total produced
2. **Demand constraint**: The sum of units sold and shortage must equal the demand

## Results Interpretation

The function returns a dictionary with:
- `optimal_production`: The optimal amount to produce (first-stage decision)
- `expected_cost`: The expected total cost
- `scenarios`: For each scenario, shows:
  - The realized demand
  - How much to sell
  - How much to store
  - How much demand goes unfulfilled (shortage)

## Key Insights

1. **Making decisions under uncertainty**: This model helps determine the optimal production quantity without knowing the exact demand.

2. **Hedging against uncertainty**: By considering multiple demand scenarios with their probabilities, the model finds a production amount that minimizes expected costs.

3. **Risk management**: The model balances the risks of:
   - Overproduction (which leads to storage costs)
   - Underproduction (which leads to shortage penalties)

4. **Value of stochastic solution**: The benefit of using this approach over a deterministic model that uses only the average demand is that it accounts for the asymmetric costs of overproduction versus underproduction.

## Running the Code

To run this code, you need:
1. Python installed with NumPy
2. Pyomo package installed
3. GLPK solver installed and accessible
4. Call the function with desired number of scenarios:
   ```python
   results = solve_two_stage_stochastic_program(num_scenarios=10)
   print(f"Optimal production: {results['optimal_production']}")
   ```

In [8]:
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import os
import sys

# Function to find the GLPK executable
def find_glpk_executable():
    """
    Find the GLPK executable path on the system.
    Returns the path if found, None otherwise.
    """
    # Potential paths for macOS
    potential_paths = [
        os.path.join(os.environ.get('CONDA_PREFIX', ''), 'bin', 'glpsol'),  # Conda path
        "/opt/homebrew/bin/glpsol",  # Homebrew on Apple Silicon
        "/usr/local/bin/glpsol",     # Homebrew on Intel Macs
        # Add Windows paths if needed
        "C:\\glpk\\w64\\glpsol.exe",
        "C:\\Program Files\\glpk\\glpsol.exe",
        "C:\\Program Files (x86)\\glpk\\glpsol.exe"
    ]
    
    for path in potential_paths:
        if os.path.exists(path):
            print(f"Found GLPK executable at: {path}")
            return path
    
    print("GLPK executable not found in expected locations.")
    return None

def solve_two_stage_stochastic_program(num_scenarios=5, seed=42):
    """
    Solve a two-stage stochastic programming problem for a simple resource allocation example.
    
    This example models a production planning problem:
    - First stage: Decide how much to produce before knowing demand
    - Second stage: After observing demand, decide how much to sell or store
    
    Parameters:
    -----------
    num_scenarios : int
        Number of demand scenarios to generate
    seed : int
        Random seed for reproducibility
    
    Returns:
    --------
    dict
        Dictionary containing the optimal production quantity, expected cost,
        and detailed information for each scenario
    """
    # Set random seed for reproducibility
    np.random.seed(seed)
    
    # Problem parameters
    production_cost = 10
    storage_cost = 2
    shortage_cost = 20
    selling_price = 15
    
    # Generate random demand scenarios
    base_demand = 100
    demand_scenarios = base_demand + np.random.normal(0, 20, num_scenarios)
    scenario_probs = np.ones(num_scenarios) / num_scenarios  # Equal probability
    
    # Create the Pyomo concrete model
    model = pyo.ConcreteModel()
    
    # First stage decision variable: how much to produce
    model.produce = pyo.Var(domain=pyo.NonNegativeReals)
    
    # Second stage variables - for each scenario
    model.scenarios = pyo.RangeSet(0, num_scenarios-1)
    model.sell = pyo.Var(model.scenarios, domain=pyo.NonNegativeReals)
    model.store = pyo.Var(model.scenarios, domain=pyo.NonNegativeReals)
    model.shortage = pyo.Var(model.scenarios, domain=pyo.NonNegativeReals)
    
    # First stage objective: minimize production cost
    model.first_stage_cost = pyo.Expression(expr=production_cost * model.produce)
    
    # Second stage objective: expected profit/cost across all scenarios
    def second_stage_rule(model, s):
        return (scenario_probs[s] * 
                (selling_price * model.sell[s] - 
                 storage_cost * model.store[s] - 
                 shortage_cost * model.shortage[s]))
    
    model.second_stage_obj = pyo.Expression(model.scenarios, rule=second_stage_rule)
    
    # Total objective: first stage cost + expected second stage cost
    model.obj = pyo.Objective(
        expr=model.first_stage_cost - sum(model.second_stage_obj[s] for s in model.scenarios),
        sense=pyo.minimize
    )
    
    # Constraints
    
    # For each scenario, amount sold + stored equals amount produced
    def balance_rule(model, s):
        return model.sell[s] + model.store[s] == model.produce
    
    model.balance = pyo.Constraint(model.scenarios, rule=balance_rule)
    
    # For each scenario, amount sold + shortage equals demand
    def demand_rule(model, s):
        return model.sell[s] + model.shortage[s] == demand_scenarios[s]
    
    model.demand = pyo.Constraint(model.scenarios, rule=demand_rule)
    
    # Set up the solver
    solver = None
    
    # Try to find GLPK executable
    glpk_path = find_glpk_executable()
    
    if glpk_path:
        # Create solver with explicit executable path
        try:
            solver = SolverFactory('glpk', executable=glpk_path)
            print("Using GLPK solver with explicit path")
        except:
            solver = None
    
    # If GLPK isn't available, try other solvers
    if solver is None:
        for solver_name in ['cbc', 'ipopt', 'glpk_direct', 'scip']:
            try:
                solver = SolverFactory(solver_name)
                if solver.available():
                    print(f"Using {solver_name} solver")
                    break
                else:
                    solver = None
            except:
                solver = None
                continue
    
    # If no solver is available, solve with a simple method instead
    if solver is None:
        print("WARNING: No solver available. Using a simple approximation method.")
        # Simplified solution: produce the average demand to minimize expected costs
        optimal_production = np.mean(demand_scenarios)
        expected_cost = production_cost * optimal_production
        
        results = {
            'optimal_production': optimal_production,
            'expected_cost': expected_cost,
            'scenarios': []
        }
        
        for s in range(num_scenarios):
            sold = min(optimal_production, demand_scenarios[s])
            stored = max(0, optimal_production - demand_scenarios[s])
            shortage = max(0, demand_scenarios[s] - optimal_production)
            
            scenario_result = {
                'scenario': s,
                'demand': demand_scenarios[s],
                'sold': sold,
                'stored': stored,
                'shortage': shortage
            }
            results['scenarios'].append(scenario_result)
            
        return results
    
    # Solve the model with the available solver
    result = solver.solve(model)
    
    # Check if the solver found a solution
    if (result.solver.status == pyo.SolverStatus.ok and 
        result.solver.termination_condition == pyo.TerminationCondition.optimal):
        # Extract and return results
        results = {
            'optimal_production': pyo.value(model.produce),
            'expected_cost': pyo.value(model.obj),
            'scenarios': []
        }
        
        for s in model.scenarios:
            scenario_result = {
                'scenario': s,
                'demand': demand_scenarios[s],
                'sold': pyo.value(model.sell[s]),
                'stored': pyo.value(model.store[s]),
                'shortage': pyo.value(model.shortage[s])
            }
            results['scenarios'].append(scenario_result)
            
        return results
    else:
        print(f"Solver status: {result.solver.status}")
        print(f"Termination condition: {result.solver.termination_condition}")
        raise Exception("The solver failed to find a solution.")


In [11]:

# Example usage
if __name__ == "__main__":
    print("Python executable:", sys.executable)
    print("PATH environment variable:", os.environ.get('PATH'))
    
    # Solve with different numbers of scenarios
    for num_scenarios in [100, 500]:
        print(f"\nSolving with {num_scenarios} scenarios:")
        try:
            results = solve_two_stage_stochastic_program(num_scenarios=num_scenarios)
            print(f"Optimal production quantity: {results['optimal_production']:.2f}")
            print(f"Expected total cost: {results['expected_cost']:.2f}")
            
            # Print details for first 3 scenarios
            for i, scenario in enumerate(results['scenarios'][:3]):
                print(f"\nScenario {i}:")
                print(f"  Demand: {scenario['demand']:.2f}")
                print(f"  Sold: {scenario['sold']:.2f}")
                print(f"  Stored: {scenario['stored']:.2f}")
                print(f"  Shortage: {scenario['shortage']:.2f}")
        except Exception as e:
            print(f"Error: {str(e)}")
            print("Trying alternative approach...")
            
            # If the main approach fails, try a direct approach with the solver
            try:
                # Print information about available solvers
                print("\nChecking available solvers:")
                for solver_name in ['glpk', 'cbc', 'ipopt', 'glpk_direct']:
                    solver = SolverFactory(solver_name)
                    print(f"Solver {solver_name} available: {solver.available()}")
            except:
                print("Could not check solvers.")

Python executable: /Users/muntazirabidi/miniconda3/envs/ai_learning/bin/python
PATH environment variable: /Users/muntazirabidi/miniconda3/envs/ai_learning/bin:/Users/muntazirabidi/.nvm/versions/node/v18.20.4/bin:/Users/muntazirabidi/miniconda3/bin:/Users/muntazirabidi/miniconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/TeX/texbin:/Users/muntazirabidi/.local/bin:/Users/muntazirabidi/.nvm/versions/node/v18.20.4/bin:/Users/muntazirabidi/miniconda3/bin:/Users/muntazirabidi/miniconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/ru

In [12]:
results

{'optimal_production': 108.25562,
 'expected_cost': -239.5357137140868,
 'scenarios': [{'scenario': 0,
   'demand': 109.93428306022466,
   'sold': 108.25562,
   'stored': 0.0,
   'shortage': 1.6786645},
  {'scenario': 1,
   'demand': 97.2347139765763,
   'sold': 97.234714,
   'stored': 11.020905,
   'shortage': 0.0},
  {'scenario': 2,
   'demand': 112.95377076201385,
   'sold': 108.25562,
   'stored': 0.0,
   'shortage': 4.6981522},
  {'scenario': 3,
   'demand': 130.46059712816052,
   'sold': 108.25562,
   'stored': 0.0,
   'shortage': 22.204979},
  {'scenario': 4,
   'demand': 95.31693250553329,
   'sold': 95.316933,
   'stored': 12.938686,
   'shortage': 0.0},
  {'scenario': 5,
   'demand': 95.3172608610164,
   'sold': 95.317261,
   'stored': 12.938358,
   'shortage': 0.0},
  {'scenario': 6,
   'demand': 131.58425631014782,
   'sold': 108.25562,
   'stored': 0.0,
   'shortage': 23.328638},
  {'scenario': 7,
   'demand': 115.34869458305818,
   'sold': 108.25562,
   'stored': 0.0,
   

In [4]:
import pyomo.environ as pyo
solvers = pyo.SolverFactory.__dict__['_cls'].keys()
print("Available solvers:", list(solvers))

Available solvers: ['_neos', 'cbc', '_cbc_shell', '_mock_cbc', 'glpk', '_glpk_shell', '_mock_glpk', 'cplex', '_cplex_shell', '_mock_cplex', 'gurobi_direct', 'asl', '_mock_asl', 'gurobi', '_gurobi_nl', '_gurobi_shell', '_gurobi_file', 'baron', 'py', 'scip', 'conopt', 'xpress', 'ipopt', 'gurobi_persistent', 'cplex_direct', 'cplex_persistent', 'gams', '_gams_direct', '_gams_shell', 'mosek', 'mosek_direct', 'mosek_persistent', 'xpress_direct', 'xpress_persistent', 'sas', '_sas94', '_sascas', 'cp_optimizer', 'mpec_nlp', 'mpec_minlp', 'path', 'appsi_gurobi', 'appsi_cplex', 'appsi_ipopt', 'appsi_cbc', 'appsi_highs', 'appsi_maingo', 'gdpopt', 'gdpopt.gloa', 'gdpopt.lbb', 'gdpopt.loa', 'gdpopt.ric', 'gdpopt.enumerate', 'gdpopt.ldsda', 'contrib.gjh', 'mindtpy', 'mindtpy.oa', 'mindtpy.ecp', 'mindtpy.goa', 'mindtpy.fp', 'multistart', 'cyipopt', 'scipy.fsolve', 'scipy.root', 'scipy.newton', 'scipy.secant-newton', 'ipopt_v2', 'gurobi_v2', 'gurobi_direct_v2', 'trustregion']
